In [32]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [33]:
from langchain_community.document_loaders import DirectoryLoader

#loader = TextLoader("./bella_vista.txt")
DATA_PATH = "mydata"
loader = DirectoryLoader(DATA_PATH, glob="*.pdf")
docs = loader.load()

#print(docs)
#print(len(docs))

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
)
documents = text_splitter.split_documents(docs)

for doc in documents:
    print("\n")
    print(doc)

len(documents)



page_content='PUSH POSSIBLETM\n\nWAREHOUSE MANAGEMENT AND TRANSPORT EXECUTION FOR IBM i\n\nRELEASE NOTES 2024\n\n0' metadata={'source': 'mydata/WMI-RN.pdf'}


page_content='0\n\nWMi AND TEi RELEASE NOTES\n\nCopyright and Disclaimer' metadata={'source': 'mydata/WMI-RN.pdf'}


page_content='Copyright © 2024 Manhattan Associates, Inc. All rights reserved.' metadata={'source': 'mydata/WMI-RN.pdf'}


page_content='This documentation, as well as the software described in it, is furnished under license and may be' metadata={'source': 'mydata/WMI-RN.pdf'}


page_content='license and may be used or copied only in accordance with the terms of such license. The' metadata={'source': 'mydata/WMI-RN.pdf'}


page_content='such license. The information in this documentation is furnished for informational use only, is' metadata={'source': 'mydata/WMI-RN.pdf'}


page_content='use only, is subject to change without notice, and should not be construed as a commitment by' metadata={'source': 'mydata/WMI-

304

In [35]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [40]:
from langchain.vectorstores.faiss import FAISS

vectorstore = FAISS.from_documents(documents, embeddings)
vectorstore.save_local("index") # newer FAISS versions can not be serialized with pickle

# Argument 'allow_dangerous_deserialization' was added due to people adding malware in pkl files -> no worry
FAISS.load_local("index", embeddings,allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()

retriever = vectorstore.as_retriever(search_kwargs={"filter": {'source': 'mydata/WMI-RN.pdf'}, "k":3})
docs = retriever.get_relevant_documents(query="does WMI support json interfaces?")

#docs = retriever.invoke(input="does WMI support json interfaces?", filter={'source': '/mydata/WMI-RN.pdf'}, k=3)

print(len(docs))

for doc in docs:
    print("\n")
    print(doc)

3


page_content='JSON Support for Interfaces' metadata={'source': 'mydata/WMI-RN.pdf'}


page_content='of data from external sources into WMi. Similarly, leveraging JSON in outbound interfaces enables' metadata={'source': 'mydata/WMI-RN.pdf'}


page_content='3 JSON Support for Interfaces' metadata={'source': 'mydata/WMI-RN.pdf'}


In [41]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """You are a helpful assistant for our Supply Chain System.

{context}

Question: {question}
Answer here:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

llm = ChatOpenAI()
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
)

result = qa.invoke(input="does WMI support json interfaces?")
print("\n")
print(result)



{'query': 'does WMI support json interfaces?', 'result': 'Yes, WMI does support JSON interfaces for importing and exporting data from external sources.'}
